## Extract Spectral Libraries from Imagery
This code uses a reference polygon dataset of plant species to extract spectra from imagery. 

* Can extract spectra from single image or multiple images in a directory. If using a directory, image spectra can be saved separately or as a single file. 
* Code can be run without metadata, metadata provided as .csv, and metadata from shapefile

Susan Meerdink  
April 2019

In [1]:
# Import Modules
import glob
import numpy as np
import rasterio.plot
from rasterio import features
from matplotlib.collections import PatchCollection
import matplotlib.pyplot as plt
import fiona
from descartes import PolygonPatch
from shapely.geometry import shape, MultiPolygon
from shapely.ops import transform
import pyproj
from functools import partial
import random
import gc
from os.path import basename
import os
import utils

In [5]:
# # Processing Single file without metadata
# polyLocation = 'C:\\Users\\susanmeerdink\\Documents\\Git\\extract-image-spectral-libraries\\Testing\\Input\\shapefile'
# imageLocation = 'C:\\Users\\susanmeerdink\\Documents\\Git\\extract-image-spectral-libraries\\Testing\\Input\\TestImage'
# outLocation = 'C:\\Users\\susanmeerdink\\Documents\\Git\\extract-image-spectral-libraries\\Testing\\Output\\output'
# create_spec_lib(polyLocation, imageLocation, outLocation, 0, 0)

In [6]:
# WHISPERS Conference 2019 - 6 Target with ACE proportions
polyLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Validation_polygons\\SB_092014_VALID'
imageLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Imagery\\ACE\\f140829t01p00r09_ACE_Alpha005_Conf_noopt_rot_geo'
outLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Imagery\\ACE\\f140829t01p00r09_ACE_Alpha005_Conf_noopt_rot_geo_fractions.csv'
create_spec_lib(polyLocation, imageLocation, outLocation, 0)

imageLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Imagery\\ACE\\f140829t01p00r10_ACE_Alpha005_Conf_noopt_rot_geo'
outLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Imagery\\ACE\\f140829t01p00r10_ACE_Alpha005_Conf_noopt_rot_geo_fractions.csv'
create_spec_lib(polyLocation, imageLocation, outLocation, 0)

# imageLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\MESMA\\18m_mos_nan_SA'
# outLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\MESMA\\18m_mos_nan_SA_fractions.csv'
# create_spec_lib(polyLocation, imageLocation, outLocation, 0)

67 Reference Polygons Found
Extracting spectra fromf140829t01p00r09_ACE_Alpha005_Conf_noopt_rot_geo
Completed processing
67 Reference Polygons Found
Extracting spectra fromf140829t01p00r10_ACE_Alpha005_Conf_noopt_rot_geo
Completed processing


In [19]:
# # WHISPERS Conference 2019 - 6 Target
polyLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Validation_polygons\\SB_092014_VALID'
imageLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Imagery\\Original\\f140829t01p00r09_corr_v1_urb_10000_SMA_20190514T15H45M47S_shadenorm_rot_geo'
outLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Imagery\\Original\\f140829t01p00r09_corr_v1_urb_10000_SMA_20190514T15H45M47S_shadenorm_rot_geo_fractions.csv'
create_spec_lib(polyLocation, imageLocation, outLocation, 0)

# imageLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Imagery\\Original\\f140829t01p00r10_corr_v1_urb_10000_SMA_20190515T00H47M22S_shadenorm_rot_geo'
# outLocation = 'C:\\Users\\susanmeerdink\\Dropbox (UFL)\\Analysis\\WHISPERS_2019\\Imagery\\Original\\f140829t01p00r10_corr_v1_urb_10000_SMA_20190515T00H47M22S_shadenorm_rot_geo_fractions.csv'
# create_spec_lib(polyLocation, imageLocation, outLocation, 0)


67 Reference Polygons Found
Extracting spectra fromf140829t01p00r09_corr_v1_urb_10000_SMA_20190514T15H45M47S_shadenorm_rot_geo
['ORIG_FID', 'Turfgrass', 'NPV', 'Paved', 'Roof', 'Soil', 'Tree', 'Uncl']
ORIG_FID
0
1
[1.         0.         0.12461633 0.79413751 0.         0.07955801
 0.00168815 0.        ]
[1.         0.         0.12461633 0.79413751 0.         0.07955801
 0.00168815 0.        ]
[1.         0.         0.12461633 0.79413751 0.         0.07955801
 0.00168815 0.        ]
[1.         0.         0.12461633 0.79413751 0.         0.07955801
 0.00168815 0.        ]
[1.         0.         0.12461633 0.79413751 0.         0.07955801
 0.00168815 0.        ]
[1.         0.         0.12461633 0.79413751 0.         0.07955801
 0.00168815 0.        ]
[1.         0.         0.12461633 0.79413751 0.         0.07955801
 0.00168815 0.        ]
[1.         0.         0.12461633 0.79413751 0.         0.07955801
 0.00168815 0.        ]
[1.         0.         0.12461633 0.79413751 0.         0.

3
[3.00000000e+00 3.55125840e-02 1.49754451e-01 3.03345611e-01
 2.10926949e-01 0.00000000e+00 2.99570289e-01 8.90117000e-04]
[3.00000000e+00 3.55125840e-02 1.49754451e-01 3.03345611e-01
 2.10926949e-01 0.00000000e+00 2.99570289e-01 8.90117000e-04]
[3.00000000e+00 3.55125840e-02 1.49754451e-01 3.03345611e-01
 2.10926949e-01 0.00000000e+00 2.99570289e-01 8.90117000e-04]
[3.00000000e+00 3.55125840e-02 1.49754451e-01 3.03345611e-01
 2.10926949e-01 0.00000000e+00 2.99570289e-01 8.90117000e-04]
[3.00000000e+00 3.55125840e-02 1.49754451e-01 3.03345611e-01
 2.10926949e-01 0.00000000e+00 2.99570289e-01 8.90117000e-04]
[3.00000000e+00 3.55125840e-02 1.49754451e-01 3.03345611e-01
 2.10926949e-01 0.00000000e+00 2.99570289e-01 8.90117000e-04]
[3.00000000e+00 3.55125840e-02 1.49754451e-01 3.03345611e-01
 2.10926949e-01 0.00000000e+00 2.99570289e-01 8.90117000e-04]
[3.00000000e+00 3.55125840e-02 1.49754451e-01 3.03345611e-01
 2.10926949e-01 0.00000000e+00 2.99570289e-01 8.90117000e-04]
[3.00000000e+0

[5.         0.01853898 0.06568447 0.48314917 0.13084714 0.02971148
 0.26531615 0.00675261]
[5.         0.01853898 0.06568447 0.48314917 0.13084714 0.02971148
 0.26531615 0.00675261]
[5.         0.01853898 0.06568447 0.48314917 0.13084714 0.02971148
 0.26531615 0.00675261]
[5.         0.01853898 0.06568447 0.48314917 0.13084714 0.02971148
 0.26531615 0.00675261]
6
10
11
12
13
15
[1.50000000e+01 2.81982812e-01 9.05770410e-02 1.60435850e-01
 7.90976060e-02 2.42786990e-02 3.61939840e-01 1.68815200e-03]
[1.50000000e+01 2.81982812e-01 9.05770410e-02 1.60435850e-01
 7.90976060e-02 2.42786990e-02 3.61939840e-01 1.68815200e-03]
[1.50000000e+01 2.81982812e-01 9.05770410e-02 1.60435850e-01
 7.90976060e-02 2.42786990e-02 3.61939840e-01 1.68815200e-03]
[1.50000000e+01 2.81982812e-01 9.05770410e-02 1.60435850e-01
 7.90976060e-02 2.42786990e-02 3.61939840e-01 1.68815200e-03]
[1.50000000e+01 2.81982812e-01 9.05770410e-02 1.60435850e-01
 7.90976060e-02 2.42786990e-02 3.61939840e-01 1.68815200e-03]
[1.5

[1.70000000e+01 6.87538370e-02 2.54450580e-02 3.53529773e-01
 2.15561694e-01 3.51135670e-02 2.97759362e-01 3.83671000e-03]
[1.70000000e+01 6.87538370e-02 2.54450580e-02 3.53529773e-01
 2.15561694e-01 3.51135670e-02 2.97759362e-01 3.83671000e-03]
[1.70000000e+01 6.87538370e-02 2.54450580e-02 3.53529773e-01
 2.15561694e-01 3.51135670e-02 2.97759362e-01 3.83671000e-03]
[1.70000000e+01 6.87538370e-02 2.54450580e-02 3.53529773e-01
 2.15561694e-01 3.51135670e-02 2.97759362e-01 3.83671000e-03]
[1.70000000e+01 6.87538370e-02 2.54450580e-02 3.53529773e-01
 2.15561694e-01 3.51135670e-02 2.97759362e-01 3.83671000e-03]
[1.70000000e+01 6.87538370e-02 2.54450580e-02 3.53529773e-01
 2.15561694e-01 3.51135670e-02 2.97759362e-01 3.83671000e-03]
[1.70000000e+01 6.87538370e-02 2.54450580e-02 3.53529773e-01
 2.15561694e-01 3.51135670e-02 2.97759362e-01 3.83671000e-03]
[1.70000000e+01 6.87538370e-02 2.54450580e-02 3.53529773e-01
 2.15561694e-01 3.51135670e-02 2.97759362e-01 3.83671000e-03]
[1.70000000e+01 

[19.          0.05285451  0.09398404  0.26169429  0.25104359  0.
  0.25828729  0.08213628]
[19.          0.05285451  0.09398404  0.26169429  0.25104359  0.
  0.25828729  0.08213628]
[19.          0.05285451  0.09398404  0.26169429  0.25104359  0.
  0.25828729  0.08213628]
[19.          0.05285451  0.09398404  0.26169429  0.25104359  0.
  0.25828729  0.08213628]
[19.          0.05285451  0.09398404  0.26169429  0.25104359  0.
  0.25828729  0.08213628]
[19.          0.05285451  0.09398404  0.26169429  0.25104359  0.
  0.25828729  0.08213628]
[19.          0.05285451  0.09398404  0.26169429  0.25104359  0.
  0.25828729  0.08213628]
[19.          0.05285451  0.09398404  0.26169429  0.25104359  0.
  0.25828729  0.08213628]
[19.          0.05285451  0.09398404  0.26169429  0.25104359  0.
  0.25828729  0.08213628]
[19.          0.05285451  0.09398404  0.26169429  0.25104359  0.
  0.25828729  0.08213628]
[19.          0.05285451  0.09398404  0.26169429  0.25104359  0.
  0.25828729  0.08213628]

...

[8.60000000e+01 9.20503380e-02 1.74063843e-01 3.78330264e-01
 2.66758748e-01 1.58686310e-02 5.56476370e-02 1.72805400e-02]
[8.60000000e+01 9.20503380e-02 1.74063843e-01 3.78330264e-01
 2.66758748e-01 1.58686310e-02 5.56476370e-02 1.72805400e-02]
[8.60000000e+01 9.20503380e-02 1.74063843e-01 3.78330264e-01
 2.66758748e-01 1.58686310e-02 5.56476370e-02 1.72805400e-02]
[8.60000000e+01 9.20503380e-02 1.74063843e-01 3.78330264e-01
 2.66758748e-01 1.58686310e-02 5.56476370e-02 1.72805400e-02]
[8.60000000e+01 9.20503380e-02 1.74063843e-01 3.78330264e-01
 2.66758748e-01 1.58686310e-02 5.56476370e-02 1.72805400e-02]
[8.60000000e+01 9.20503380e-02 1.74063843e-01 3.78330264e-01
 2.66758748e-01 1.58686310e-02 5.56476370e-02 1.72805400e-02]
[8.60000000e+01 9.20503380e-02 1.74063843e-01 3.78330264e-01
 2.66758748e-01 1.58686310e-02 5.56476370e-02 1.72805400e-02]
[8.60000000e+01 9.20503380e-02 1.74063843e-01 3.78330264e-01
 2.66758748e-01 1.58686310e-02 5.56476370e-02 1.72805400e-02]
[8.60000000e+01 

[8.90000000e+01 0.00000000e+00 1.13566600e-03 6.60098220e-01
 2.15623082e-01 0.00000000e+00 1.23143033e-01 0.00000000e+00]
[8.90000000e+01 0.00000000e+00 1.13566600e-03 6.60098220e-01
 2.15623082e-01 0.00000000e+00 1.23143033e-01 0.00000000e+00]
[8.90000000e+01 0.00000000e+00 1.13566600e-03 6.60098220e-01
 2.15623082e-01 0.00000000e+00 1.23143033e-01 0.00000000e+00]
[8.90000000e+01 0.00000000e+00 1.13566600e-03 6.60098220e-01
 2.15623082e-01 0.00000000e+00 1.23143033e-01 0.00000000e+00]
[8.90000000e+01 0.00000000e+00 1.13566600e-03 6.60098220e-01
 2.15623082e-01 0.00000000e+00 1.23143033e-01 0.00000000e+00]
[8.90000000e+01 0.00000000e+00 1.13566600e-03 6.60098220e-01
 2.15623082e-01 0.00000000e+00 1.23143033e-01 0.00000000e+00]
[8.90000000e+01 0.00000000e+00 1.13566600e-03 6.60098220e-01
 2.15623082e-01 0.00000000e+00 1.23143033e-01 0.00000000e+00]
[8.90000000e+01 0.00000000e+00 1.13566600e-03 6.60098220e-01
 2.15623082e-01 0.00000000e+00 1.23143033e-01 0.00000000e+00]
[8.90000000e+01 

[1.001e+03 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
 0.000e+00]
[1.001e+03 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
 0.000e+00]
[1.001e+03 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
 0.000e+00]
[1.001e+03 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
 0.000e+00]
[1.001e+03 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
 0.000e+00]
[1.001e+03 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
 0.000e+00]
[1.001e+03 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
 0.000e+00]
[1.001e+03 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
 0.000e+00]
[1.001e+03 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
 0.000e+00]
[1.001e+03 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
 0.000e+00]
[1.001e+03 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
 0.000e+00]
[1.001e+03 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
 0.000e+00]
[1.0

[1.003e+03 0.000e+00 4.242e-01 4.650e-02 0.000e+00 5.238e-01 5.500e-03
 0.000e+00]
[1.003e+03 0.000e+00 4.242e-01 4.650e-02 0.000e+00 5.238e-01 5.500e-03
 0.000e+00]
[1.003e+03 0.000e+00 4.242e-01 4.650e-02 0.000e+00 5.238e-01 5.500e-03
 0.000e+00]
[1.003e+03 0.000e+00 4.242e-01 4.650e-02 0.000e+00 5.238e-01 5.500e-03
 0.000e+00]
[1.003e+03 0.000e+00 4.242e-01 4.650e-02 0.000e+00 5.238e-01 5.500e-03
 0.000e+00]
[1.003e+03 0.000e+00 4.242e-01 4.650e-02 0.000e+00 5.238e-01 5.500e-03
 0.000e+00]
[1.003e+03 0.000e+00 4.242e-01 4.650e-02 0.000e+00 5.238e-01 5.500e-03
 0.000e+00]
[1.003e+03 0.000e+00 4.242e-01 4.650e-02 0.000e+00 5.238e-01 5.500e-03
 0.000e+00]
[1.003e+03 0.000e+00 4.242e-01 4.650e-02 0.000e+00 5.238e-01 5.500e-03
 0.000e+00]
[1.003e+03 0.000e+00 4.242e-01 4.650e-02 0.000e+00 5.238e-01 5.500e-03
 0.000e+00]
[1.003e+03 0.000e+00 4.242e-01 4.650e-02 0.000e+00 5.238e-01 5.500e-03
 0.000e+00]
[1.003e+03 0.000e+00 4.242e-01 4.650e-02 0.000e+00 5.238e-01 5.500e-03
 0.000e+00]
[1.0

 0.000e+00]
[1.005e+03 0.000e+00 1.360e-01 1.059e-01 0.000e+00 7.371e-01 2.090e-02
 0.000e+00]
[1.005e+03 0.000e+00 1.360e-01 1.059e-01 0.000e+00 7.371e-01 2.090e-02
 0.000e+00]
[1.005e+03 0.000e+00 1.360e-01 1.059e-01 0.000e+00 7.371e-01 2.090e-02
 0.000e+00]
[1.005e+03 0.000e+00 1.360e-01 1.059e-01 0.000e+00 7.371e-01 2.090e-02
 0.000e+00]
[1.005e+03 0.000e+00 1.360e-01 1.059e-01 0.000e+00 7.371e-01 2.090e-02
 0.000e+00]
[1.005e+03 0.000e+00 1.360e-01 1.059e-01 0.000e+00 7.371e-01 2.090e-02
 0.000e+00]
[1.005e+03 0.000e+00 1.360e-01 1.059e-01 0.000e+00 7.371e-01 2.090e-02
 0.000e+00]
[1.005e+03 0.000e+00 1.360e-01 1.059e-01 0.000e+00 7.371e-01 2.090e-02
 0.000e+00]
[1.005e+03 0.000e+00 1.360e-01 1.059e-01 0.000e+00 7.371e-01 2.090e-02
 0.000e+00]
[1.005e+03 0.000e+00 1.360e-01 1.059e-01 0.000e+00 7.371e-01 2.090e-02
 0.000e+00]
[1.005e+03 0.000e+00 1.360e-01 1.059e-01 0.000e+00 7.371e-01 2.090e-02
 0.000e+00]
[1.005e+03 0.000e+00 1.360e-01 1.059e-01 0.000e+00 7.371e-01 2.090e-02
 0.0

In [9]:
# # Processing single file with shapefile metadata
# polyLocation = 'C:\\Users\\susanmeerdink\\Documents\\Git\\extract-image-spectral-libraries\\Testing\\Input\\shapefile'
# imageLocation = 'C:\\Users\\susanmeerdink\\Documents\\Git\\extract-image-spectral-libraries\\Testing\\Input\\TestImage'
# outLocation = 'C:\\Users\\susanmeerdink\\Documents\\Git\\extract-image-spectral-libraries\\Testing\\Output\\output'
# create_spec_lib(polyLocation, imageLocation, outLocation, 0)

In [10]:
# # Processing single file with .csv metadata
# polyLocation = 'C:\\Users\\susanmeerdink\\Documents\\Git\\extract-image-spectral-libraries\\Testing\\Input\\shapefile'
# imageLocation = 'C:\\Users\\susanmeerdink\\Documents\\Git\\extract-image-spectral-libraries\\Testing\\Input\\TestImage'
# outLocation = 'C:\\Users\\susanmeerdink\\Documents\\Git\\extract-image-spectral-libraries\\Testing\\Output\\output'
# metaLocation = 'C:\\Users\\susanmeerdink\\Documents\\Git\\extract-image-spectral-libraries\\Testing\\Input\\meta.csv'
# create_spec_lib(polyLocation, imageLocation, outLocation, 0, [metaLocation, '', ''])

In [11]:
# Processing multiple files without metadata separately

In [12]:
# Processing multiple files with shapefile metadata separately

In [13]:
# Processing multiple files with .csv metadata separately

In [14]:
# Processing multiple files without metadata together

In [15]:
# Processing multiple files with shapefile metadata together

In [16]:
# Processing multiple files with .csv metadata together